### *IT3052E - Fundamentals of Optimization*
# **Mini Project 18 - Nurse Scheduling Problem**
#### **Techniques used**:
* Backtracking,
* Constraint Programming,
* Linear Programming,
* Local Search, and
* Meta-heuristics (Genetic Algorithm).

#### * ***Import pandas for printing solution***

In [ ]:
import pandas as pd
import time

## 3. Hill-climbing search with PyCBLS

### 3.1. Import modules and libraries

In [ ]:
from PyCBLS.VarIntLS import VarIntLS
from PyCBLS.LocalSearchManager import LocalSearchManager
from PyCBLS.NotEqual import NotEqual
from PyCBLS.ConstraintSystem import ConstraintSystem
from PyCBLS.HillClimbingSearch import HillClimbingSearch
from PyCBLS.ConditionalSumWithBound import ConditionalSumWithBound
import random as rd

### 3.2. Read data from files

In [ ]:
with open('SampleData/testCase2/8.txt') as file:
  N, D, a, b = [int(q) for q in file.readline().split()]
  dayoff = [[0 for d in range(D)] for n in range(N)]
  for n in range(N):
    for d in [int(h) for h in file.readline().split()]:
      if d != -1:
            dayoff[n][d-1] = 1
            
starttime = time.time()

### 3.3. Generate local search and define decision variable

In [ ]:
mgr = LocalSearchManager()

In [ ]:
assign = [[VarIntLS(mgr,0,4,rd.randint(1, 4),f'assign_{n}_{d}') for n in range(N)] for d in range(D)]

mor = VarIntLS(mgr,1,1,1,'one')
aft = VarIntLS(mgr,2,2,2,'one')
eve = VarIntLS(mgr,3,3,3,'one')
nig = VarIntLS(mgr,4,4,4,'one')
zero = VarIntLS(mgr,0,0,0,'zero')

### 3.4. Generate constraints

In [ ]:
constraints = []

#### 3.4.1. A nurse can be assigned to only one shift per day.

This is constrained by creating decision variable

#### 3.4.2. Each shift has min $a$ nurses and max $b$ nurses.

In [ ]:
count = [1 for i in range(N)]
for d in range(D):
    c = ConditionalSumWithBound(assign[d],count,1,a,b,'CountMor')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,2,a,b,'CountAft')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,3,a,b,'CountEve')
    constraints.append(c)
    c = ConditionalSumWithBound(assign[d],count,4,a,b,'CountNig')
    constraints.append(c)

#### 3.4.3. Set the given dayoff and dayoff after a night shift

In [ ]:
for n in range(N):
    for d in range(D-1):
        if dayoff[n][d] == 1 or assign[d-1][n] == nig:
            c = NotEqual(assign[d][n],mor,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],aft,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],eve,'NotEqual')
            constraints.append(c)
            c = NotEqual(assign[d][n],nig,'NotEqual')
            constraints.append(c)
        else:
            c = NotEqual(assign[d][n],zero,'NotEqual')
            constraints.append(c)
            
    # handle the last day
    if dayoff[n][D-1] == 1:
        c = NotEqual(assign[D-1][n],mor,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],aft,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],eve,'NotEqual')
        constraints.append(c)
        c = NotEqual(assign[D-1][n],nig,'NotEqual')
        constraints.append(c)
    else:
        c = NotEqual(assign[D-1][n],zero,'NotEqual')
        constraints.append(c)

### 3.5. Start local search

In [ ]:
C = ConstraintSystem(constraints)
mgr.close()

print('Init, C = ',C.violations())
searcher = HillClimbingSearch(C)
searcher.search(5000)

### 3.6. Print the schedule

In [ ]:
res = [[0 for n in range(N)] for d in range(D)]
for d in range(D):
    for n in range(N):
        res[d][n] = assign[d][n].getValue()
df = pd.DataFrame(res, index = [d+1 for d in range(D)], columns = [n+1 for n in range(N)])
df.index.name = 'Nurse'
df.columns.name = 'Day'
display(df)
print('Running time:',time.time() - starttime)

### 3.7. Optimal solution

In [ ]:
maxNightShift = -1
for n in range(N):
    tmp = 0
    for d in range(D):
        if res[d][n] == 4:
            tmp += 1
    if tmp > maxNightShift:
        maxNightShift = tmp
        
print('Optimal solution - Max night shift assigned to a nurse:', maxNightShift)